# Number of Islands

Given an m x n 2D binary grid grid which represents a map of '1's (land) and '0's (water), return the number of islands.

An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

Link: https://leetcode.com/problems/number-of-islands/description/

In [1]:
import networkx
from typing import Hashable

In [2]:
class DiGraph:
    """A directed graph with hashable node objects.

    Edges are between different nodes.
    There's at most one edge from one node to another.
    """

    def __init__(self):
        self.out = dict()   # a map of nodes to their out-neighbours

    def has_node(self, node: Hashable) -> bool:
        """Return True if and only if the graph has the node."""
        return node in self.out

    def has_edge(self, start: Hashable, end: Hashable) -> bool:
        """Return True if and only if edge start -> end exists.

        Preconditions: self.has_node(start) and self.has_node(end)
        """
        return end in self.out[start]

    def add_node(self, node: Hashable) -> None:
        """Add the node to the graph.

        Preconditions: not self.has_node(node)
        """
        self.out[node] = set()

    def add_edge(self, start: Hashable, end: Hashable) -> None:
        """Add edge start -> end to the graph.

        If the edge already exists, do nothing.

        Preconditions:
        self.has_node(start) and self.has_node(end) and start != end
        """
        self.out[start].add(end)

    def remove_node(self, node: Hashable) -> None:
        """Remove the node and all its attached edges.

        Preconditions: self.has_node(node)
        """
        for start in self.out:
            self.remove_edge(start, node)
        self.out.pop(node)

    def remove_edge(self, start: Hashable, end: Hashable) -> None:
        """Remove edge start -> end from the graph.

        If the edge doesn't exist, do nothing.

        Preconditions: self.has_node(start) and self.has_node(end)
        """
        self.out[start].discard(end)

    def nodes(self) -> set:
        """Return the graph's nodes."""
        all_nodes = set()
        for node in self.out:
            all_nodes.add(node)
        return all_nodes

    def edges(self) -> set:
        """Return the graph's edges as a set of pairs (start, end)."""
        all_edges = set()
        for start in self.out:
            for end in self.out[start]:
                all_edges.add( (start, end) )
        return all_edges

    def out_neighbours(self, node: Hashable) -> set:
        """Return the out-neighbours of the node.

        Preconditions: self.has_node(node)
        """
        return set(self.out[node])  # return a copy

    def out_degree(self, node: Hashable) -> int:
        """Return the number of out-neighbours of the node.

        Preconditions: self.has_node(node)
        """
        return len(self.out[node])

    def in_neighbours(self, node: Hashable) -> set:
        """Return the in-neighbours of the node.

        Preconditions: self.has_node(node)
        """
        start_nodes = set()
        for start in self.out:
            if self.has_edge(start, node):
                start_nodes.add(start)
        return start_nodes

    def in_degree(self, node: Hashable) -> int:
        """Return the number of in-neighbours of the node.

        Preconditions: self.has_node(node)
        """
        return len(self.in_neighbours(node))

    def neighbours(self, node: Hashable) -> set:
        """Return the in- and out-neighbours of the node.

        Preconditions: self.has_node(node)
        """
        return self.out_neighbours(node).union(self.in_neighbours(node))

    def degree(self, node: Hashable) -> int:
        """Return the number of in- and out-going edges of the node.

        Preconditions: self.has_node(node)
        """
        return self.in_degree(node) + self.out_degree(node)

        

class UndirectedGraph(DiGraph):
    """An undirected graph with hashable node objects.

    There's at most one edge between two different nodes.
    There are no edges between a node and itself.
    """

    def add_edge(self, node1: Hashable, node2: Hashable) -> None:
        """Add an undirected edge node1-node2 to the graph.

        If the edge already exists, do nothing.

        Preconditions: self.has_node(node1) and self.has_node(node2)
        """
        super().add_edge(node1, node2)
        super().add_edge(node2, node1)

    def remove_edge(self, node1: Hashable, node2: Hashable) -> None:
        """Remove edge node1-node2 from the graph.

        If the edge doesn't exist, do nothing.

        Preconditions: self.has_node(node1) and self.has_node(node2)
        """
        super().remove_edge(node1, node2)
        super().remove_edge(node2, node1)

    def edges(self) -> set:
        """Return the graph's edges as a set of pairs.

        Postconditions: for every edge A-B,
        the output has either (A, B) or (B, A) but not both
        """
        all_edges = set()
        for node1 in self.out:
            for node2 in self.out[node1]:
                if (node2, node1) not in all_edges:
                    all_edges.add( (node1, node2) )
        return all_edges

    def in_neighbours(self, node: Hashable) -> set:
        """Return all nodes that are adjacent to the node.

        Preconditions: self.has_node(node)
        """
        return self.out_neighbours(node)

    def neighbours(self, node: Hashable) -> set:
        """Return all nodes that are adjacent to the node.

        Preconditions: self.has_node(node)
        """
        return self.out_neighbours(node)

    def in_degree(self, node: Hashable) -> int:
        """Return the number of edges attached to the node.

        Preconditions: self.has_node(node)
        """
        return self.out_degree(node)

    def degree(self, node: Hashable) -> int:
        """Return the number of edges attached to the node.

        Preconditions: self.has_node(node)
        """
        return self.out_degree(node)
    
    def draw(self) -> None:
        """Draw the graph."""
        if type(self) == DiGraph:
            graph = networkx.DiGraph()
        else:
            graph = networkx.Graph()
        graph.add_nodes_from(self.nodes())
        graph.add_edges_from(self.edges())
        networkx.draw(graph, with_labels=True,
            node_size=1000, node_color='lightblue',
            font_size=12, font_weight='bold')


In [6]:
class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        undirected = UndirectedGraph()
        ## add nodes
        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] == "1":
                    node = f"{i}.{j}"
                    undirected.add_node(node)
                    
        ## add edges
        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] == "1":
                    node = f"{i}.{j}"
                    if i-1 >= 0 and grid[i-1][j] == "1":
                        undirected.add_edge(node, f"{i-1}.{j}")
                    if i+1 < len(grid) and grid[i+1][j] == "1":
                        undirected.add_edge(node, f"{i+1}.{j}")
                    if j-1 >= 0 and grid[i][j-1] == "1":
                        undirected.add_edge(node, f"{i}.{j-1}")
                    if j+1 < len(grid[i]) and grid[i][j+1] == "1":
                        undirected.add_edge(node, f"{i}.{j+1}")
        

        connected_components = self.connected_components(undirected)
        return max([v for k, v in connected_components.items()])
    
    def dfs(self,graph: DiGraph, start: Hashable) -> DiGraph:
        """Return the subgraph traversed by a depth-first search.

        Preconditions: graph.has_node(start)
        """
        visited = DiGraph()
        visited.add_node(start)
        unprocessed = []                            # deque -> list
        for neighbour in graph.out_neighbours(start):
            unprocessed.append( (start, neighbour) )
        while len(unprocessed) > 0:
            edge = unprocessed.pop()                # popleft -> pop
            previous = edge[0]
            current = edge[1]
            if not visited.has_node(current):
                visited.add_node(current)
                visited.add_edge(previous, current)
                for neighbour in graph.out_neighbours(current):
                    unprocessed.append( (current, neighbour) )
        return visited
    
    def connected_components(self, graph: UndirectedGraph) -> dict:
        """Return the connected components of graph.

        Postconditions: the output maps each node to its component,
        numbered from 1 onwards.
        """
        component = dict()
        counter = 1
        for node in graph.nodes():
            if node not in component:
                tree = self.dfs(graph, node)
                for reached in tree.nodes():
                    component[reached] = counter
                counter = counter + 1
        return component

In [7]:
grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]

s = Solution().numIslands(grid)
print(s)

1


In [8]:
grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
s = Solution().numIslands(grid)
print(s)

3
